## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 아산시청 위도, 경도
Asan = [36.789882248764656, 127.00274491353838]

# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 아산시 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2405_001/_census_data_2023_bnd_dong_bnd_dong_34040_2023_2023/bnd_dong_34040_2023_2023_2Q.shp"
asan_gdf = gpd.read_file(shapefile_path)
asan_gdf = asan_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 행정동 구분에 따라 색 구분하는 함수
def hjd_color(name):
    if name[-1] == '읍':
        return 'green'
    elif name[-1] == '면':
        return 'yellow'
    elif name[-1] == '동':
        return 'red'

asan_gdf['color'] = asan_gdf['ADM_NM'].apply(hjd_color)

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 필터링된 원도심 grid(격자) 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
for _, row in asan_gdf.iterrows() :
    color = row['color']
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature, color=color:{
            'fillColor': color,
            'color': 'black',
            'weight': 1
        }).add_to(m)
# 맵 출력
m

#### 격자(매핑용)

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/12.아산시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 필터링된 원도심 grid(격자) 추가

for _, row in asan_gdf.iterrows() :
    color = row['color']
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature, color=color:{
            'fillColor': color,
            'color': 'black',
            'weight': 1
        }).add_to(m)

folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = grid_map_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 0.3
    }
)
grid_layer.add_to(m)
# 맵 출력
m

#### 아산시 상세 도로망

In [6]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/4.아산시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [ ]:
from shapely.ops import unary_union
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_layer = folium.GeoJson(
    unary_union(asan_gdf['geometry']),
    name="아산시 행정동",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)
roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)
# 맵 출력
m

#### 아산시 질병분류별 진료현황

In [5]:
# 질병분류별 진료현황 데이터프레임
present_condition_df = pd.read_csv('SBJ_2405_001/20.아산시_질병중분류별진료현황.csv', engine='python')
# 질병코드 : 질병명 딕셔너리 생성
dss_dic = dict(zip(present_condition_df['dss_gbn'], present_condition_df['dss_nm']))

#### 아산시 AED위치정보

In [6]:
AED_df = pd.read_csv('SBJ_2405_001/8.아산시_자동심장충격기(AED)위치정보.csv')
AED_df = geo_transform(AED_df)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 병원정보

In [7]:
# 병원 데이터프레임을 GeoPandas 데이터프레임으로 변환
hospital_df = pd.read_csv('SBJ_2405_001/21.아산시_병원정보.csv')
hospital_df = geo_transform(hospital_df)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 병원진료과목정보

In [14]:
# 병원 데이터프레임을
treatment_df = pd.read_csv('SBJ_2405_001/22.아산시_병원진료과목정보.csv')

#### 아산시 약국현황

In [16]:
# 약국 데이터프레임을 GeoPandas 데이터프레임으로 변환
pharmacy_df = pd.read_csv('SBJ_2405_001/23.아산시_약국현황.csv')
pharmacy_df = geo_transform(pharmacy_df)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 도로명주소(건물)

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/11.아산시_도로명주소(건물).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_building_df = pd.json_normalize(geojson_data['features'])
roadname_building_df = roadname_building_df[['properties.BD_MGT_SN', 'properties.RN_CD', 'properties.GRO_FLO_CO', 'properties.UND_FLO_CO', 'geometry.coordinates']]
roadname_building_df['properties.GRO_FLO_CO'] = roadname_building_df['properties.GRO_FLO_CO'].astype(int)
roadname_building_df['properties.UND_FLO_CO'] = roadname_building_df['properties.UND_FLO_CO'].astype(int)
roadname_building_df['geometry'] = roadname_building_df['geometry.coordinates'].apply(lambda x : make_pol(x))

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
roadname_building_df = gpd.GeoDataFrame(roadname_building_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=14)

# 층수로 색 구분하는 함수 설정
def color_picker(floor):
    if floor > 8:
        return 'red'
    elif 8 > floor >= 4:
        return 'yellow'
    elif 4 > floor:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     top: 50px; right: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 1);
     "> &nbsp; 건물높이 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 8층이상 <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 4층이상 <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 4층이하
     </div>
     """

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'blue',
            'weight': 1
        }).add_to(m)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in roadname_building_df.iterrows():
    popup_text = f"지상:{row['properties.GRO_FLO_CO']}층, 지하:{row['properties.UND_FLO_CO']}층)"
    color = color_picker(row['properties.GRO_FLO_CO'])
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'color': color, 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

## 시설위치 시각화

#### 통합 MAP 제작

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = tot_roadsystem['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 인도 추가
m_sidewalk = folium.FeatureGroup(name="인도", overlay=True)
for _, row in filtered_sidewalk.iterrows() :
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature: {
            'fillColor': 'blue',
            'color': 'blue',
            'weight': 1
        }).add_to(m_sidewalk)
m_sidewalk.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)

# 면적으로 radius(반지름)을 계산하는 함수 정의
import math
def find_radius_from_area(area):
    radius = math.sqrt(area / math.pi)
    return radius
# 시장 위치 매핑 및 find_radius_from_area 함수로 시장 면적만큼 radius 설정
m_market = folium.FeatureGroup(name="시장", overlay=True)
for _, row in filtered_points_market.iterrows() :
    popup_text = f"{row['market_nm']} - {row['addr']}"
    folium.Circle(location=(row['lat'], row['lon']), radius=find_radius_from_area(row['area']), color='red', weight = 1,
            fill='red', name="시장").add_to(m_market).add_child(folium.Popup(popup_text, max_width=200))
m_market.add_to(m)

# 주차장 위치 매핑
m_parking = folium.FeatureGroup(name="주차장", overlay=True)
for _, row in filtered_points_parking.iterrows() :
    popup_text = f"{row['carpark_nm']} - {row['slots']}대"
    folium.Marker(location=(row['lat'], row['lon']),
    icon = folium.CustomIcon(icon_image='icon/주차장.png', icon_size=(20, 20))).add_to(m_parking).add_child(folium.Popup(popup_text, max_width=200))
m_parking.add_to(m)

# 공원 위치 매핑
m_park = folium.FeatureGroup(name="공원", overlay=True)
for _, row in filtered_points_park.iterrows() :
    popup_text = f"{row['park_nm']}({row['park_gbn']})"
    folium.Circle(location=(row['lat'], row['lon']), radius=50, color='green', weight = 1,
            fill='red', name="공원").add_to(m_park).add_child(folium.Popup(popup_text, max_width=200))
m_park.add_to(m)

# 가로수 위치 매핑
m_tree = folium.FeatureGroup(name="가로수", overlay=True)
for _, row in filtered_points_tree.iterrows() :
    popup_text = f"{row['수종명']}"
    folium.Marker(location=(row['lat'], row['lon']),
    icon = folium.CustomIcon(icon_image='icon/가로수.png', icon_size=(20, 20))).add_to(m_tree).add_child(folium.Popup(popup_text, max_width=200))
m_tree.add_to(m)

# 문화재 위치 매핑
m_culture = folium.FeatureGroup(name="문화재", overlay=True)
for _, row in filtered_points_culture.iterrows() :
    popup_text = f"{row['clt_nm']}"
    folium.Marker(location=(row['lat'], row['lon']),
    icon = folium.CustomIcon(icon_image='icon/문화재.png', icon_size=(20, 20))).add_to(m_culture).add_child(folium.Popup(popup_text, max_width=200))
m_culture.add_to(m)

# 공공기관 매핑
for public_name in filtered_points_public['inst_gbn'].unique():
    m_public = folium.FeatureGroup(name=f"{public_name}", overlay=True)
    subset = filtered_points_public[filtered_points_public['inst_gbn'] == public_name]
    for _, row in subset.iterrows() :
        popup_text = f"{row['inst_nm']}({row['inst_gbn']})"
        folium.Marker(location=(row['lat'], row['lon']),
        icon = folium.CustomIcon(icon_image=f"icon/{public_name}.png", icon_size=(20, 20))).add_to(m_public).add_child(folium.Popup(popup_text, max_width=200))
    else:
        m_public.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [ ]:
m.save("visualization/7. 원도심 시설 분석/원도심 시설위치 시각화.html")